In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import sys

ModuleNotFoundError: No module named 'open3d'

In [2]:
mesh_dir = "/media/shubodh/DATA/Downloads/data-non-onedrive/RIO10_data/scene01/models01/seq01_01/"
mesh = o3d.io.read_triangle_mesh(os.path.join(mesh_dir, "mesh.obj"), True)
o3d.visualization.draw_geometries([mesh])


[Open3D INFO] Skipping non-triangle primitive geometry of type: 2


### Convex hull

In [3]:
hull, _ = mesh.compute_convex_hull()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([mesh, hull_ls])

In [4]:
pcd_hull = o3d.geometry.PointCloud()
pcd_hull.points = hull_ls.points
pcd_hull.colors = o3d.utility.Vector3dVector(np.ones((np.asarray(hull_ls.points)).shape)*0)
o3d.visualization.draw_geometries([mesh, pcd_hull])


In [5]:
(np.asarray(hull_ls.points)).shape

(187, 3)

### dbscan clustering

In [6]:
pcd = pcd_hull

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd.cluster_dbscan(eps=0.5, min_points=2, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 19
point cloud has 19 clusters


### fixing up coordinate for clusters

In [7]:
hi = (np.asarray(pcd.points))
hi[:,2] = np.ones((hi[:,2]).shape)

In [8]:
coord_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(origin=[0, 0, 0], size=1)
coord_pcd =  coord_mesh.sample_points_uniformly(number_of_points=500)


In [9]:
sphere_mesh = o3d.geometry.TriangleMesh.create_sphere(radius=0.2)
# sphere = sphere_mesh.sample_points_uniformly(number_of_points=500)
# zero = (1.67-4.2)/2
# one = (4.05 - 0.38)/2
# two = 1
zero, one, two = np.mean(hi, axis=0)
sphere = copy.deepcopy(sphere_mesh).translate((zero, one, two))
sphere.paint_uniform_color([1, 0.706, 0])

TriangleMesh with 762 points and 1520 triangles.

In [11]:
o3d.visualization.draw_geometries([pcd, mesh, coord_mesh,sphere])